In [8]:
import pandas as pd
import json

In [17]:
file_path = "../raw_data/tickets_classification_eng.json"
with open(file_path, "r") as file:  
    datos = json.load(file)

df = pd.json_normalize(datos)

In [18]:
df.head()

,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,...,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided
0,complaint-public-v2,complaint,3211475,0.0,None,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,None,Credit card debt,Yes,,Debt is not yours,Consent not provided
1,complaint-public-v2,complaint,3229299,0.0,Servicemember,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,None,Credit card debt,Yes,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt,Consent provided
2,complaint-public-v2,complaint,3199379,0.0,None,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card,Consent provided
3,complaint-public-v2,complaint,2673060,0.0,None,48066,2673060,Trouble during payment process,2017-09-13T12:00:00-05:00,MI,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2017-09-14T12:00:00-05:00,None,Conventional home mortgage,Yes,,None,Consent not provided
4,complaint-public-v2,complaint,3203545,0.0,None,10473,3203545,Fees or interest,2019-04-05T12:00:00-05:00,NY,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2019-04-05T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,,Charged too much interest,N/A


In [ ]:
df = df[['_source.complaint_what_happened', '_source.product', '_source.sub_product']]
df = df.rename({'_source.complaint_what_happened':'complaint_what_happened',
                '_source.product':'category',
                '_source.sub_product':'sub_product'
                })
df['ticket_classification'] = df['category'] + df['sub_product']

    3. **Creación de Nueva Columna**:
       Añade una nueva columna llamada `ticket_classification` que sea el resultado de concatenar los valores de las columnas `category` y `sub_product`, separados por un signo más. Por ejemplo, si `category` contiene "Banco" y `sub_product` contiene "Cuenta Corriente", entonces `ticket_classification` debería ser "Banco + Cuenta Corriente".
    
    4. **Eliminar Columnas Redundantes**:
       Después de crear la columna `ticket_classification`, elimina las columnas `sub_product` y `category`, ya que su información ahora está encapsulada en la nueva columna.
    
    5. **Limpieza de Datos en Columnas Específicas**:
       Asegúrate de que la columna `complaint_what_happened` no contenga campos vacíos. Reemplaza esos campos vacíos con un valor que indique que los datos están ausentes (como `NaN`).
    
    6. **Eliminación de Filas con Datos Faltantes**:
       Elimina todas las filas que tengan datos faltantes en las columnas críticas, es decir, `complaint_what_happened` y `ticket_classification`.
    
    7. **Reiniciar Índice**:
       Finalmente, reinicia el índice del dataframe para asegurarte de que los índices sean consecutivos, lo cual es útil después de eliminar filas para mantener la consistencia y facilidad de acceso por índice.
    8. **Guardar el DataFrame en un Archivo CSV**:
   Guarda el DataFrame transformado en un archivo CSV. Elige un nombre de archivo que refleje el contenido del DataFrame y decide la ubicación más adecuada para guardar el archivo. Asegúrate de establecer el parámetro para no guardar el índice si no es necesario.


In [ ]:
       - `_source.complaint_what_happened`
       - `_source.product`
       - `_source.sub_product`